In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

#These are some matplot lib configurations that I like to use. 
mpl.rc("text", usetex=False) #if you have Latex set this to true and matplotlib will render labels in latex
mpl.rc("font", family = "serif") #Serif font in matplotlib
mpl.rc("figure",figsize=(9,6)) #Increase default figure size
%config InlineBackend.figure_format = 'retina' #If you have a high res display, render the plots more nicely
mpl.style.use('dark_background') #Use a dark background for matplotlib figures if youre using dark theme
plt.rcParams.update({"figure.facecolor": "111111",})

In [3]:
from scipy.integrate import solve_ivp

In [4]:
def elastic_pendulum(t,y):
    L, v, theta, w = y
    dLdt = v
    dvdt = L*w**2+ 1-L+ np.cos(theta)
    dthetadt = w
    dwdt = (-1/L)*(np.sin(theta)+2*w*v)
    return np.array([dLdt,dvdt,dthetadt,dwdt])

In [5]:
elastic_pendulum_sol = solve_ivp(elastic_pendulum,(0,100),y0=[2,0.2,1.5,-0.5],t_eval=np.linspace(0,50,5000))

In [6]:
X = elastic_pendulum_sol.y[0]*np.sin(elastic_pendulum_sol.y[2])
Y = -elastic_pendulum_sol.y[0]*np.cos(elastic_pendulum_sol.y[2])

In [7]:
# taken from the very nice blog post https://scipython.com/blog/the-spring-pendulum/
def plot_spring(L,theta):
    """Plot the spring from (0,0) to (x,y) as the projection of a helix."""
    # Spring turn radius, number of turns
    rs, ns = 0.05, 25
    # Number of data points for the helix
    Ns = 1000
    # We don't draw coils all the way to the end of the pendulum:
    # pad a bit from the anchor and from the bob by these number of points
    ipad1, ipad2 = 100, 150
    w = np.linspace(0, L, Ns)
    # Set up the helix along the x-axis ...
    xp = np.zeros(Ns)
    xp[ipad1:-ipad2] = rs * np.sin(2*np.pi * ns * w[ipad1:-ipad2] / L)
    # ... then rotate it to align with  the pendulum and plot.
    R = np.array([[np.cos(theta), -np.sin(theta)],
                  [np.sin(theta), np.cos(theta)]])
    xs, ys = - R @ np.vstack((xp, w))
    plt.plot(xs, ys, c='w', lw=2)

In [8]:
!mkdir tmp

In [10]:
for i,t in enumerate(elastic_pendulum_sol.t[:100]):
#for i in [10,100,1000,4000]:
    fig = plt.figure(figsize=(10,10))
    plt.plot(X[:i+1],Y[:i+1],alpha=0.6,c='xkcd:avocado')
    plt.xlim([-3,3])
    plt.ylim([-5,1])
    plot_spring(elastic_pendulum_sol.y[0,i],elastic_pendulum_sol.y[2,i])
    c1 = plt.Circle((X[i], Y[i]), .04, fc='r', ec='r', zorder=10)
    plt.gca().add_patch(c1)
    plt.savefig(f'tmp/img{i:04d}.png',dpi=200)
    plt.close()
    #plt.show()

`ffmpeg` is a command line utility to make movies out of images. The syntax for using it is complicated but the line below should run on your computer if you have `ffmpeg` installed. Google it to find out how to install it as it varies by operating system.

The next cell converts all the png files made in the previous cell to the movie `elastic_pendulum.mp4`.

In [ ]:
!ffmpeg -r 50 -i tmp/img%04d.png -c:v libx264 -vf fps=25 -pix_fmt yuv420p elastic_pendulum.mp4

In [ ]:
#Remove all the images that were stitched together to make the movie
!rm tmp/*.png